# The Effects of the Coronavirus on the NYC Taxi Industry

The goal of this project is to answer a number of questions about the effects of the coronavirus on travel by taxi in NYC.

# Import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, date
import warnings

warnings.filterwarnings("ignore")

In [ ]:
taxi_2019 = pd.read_csv('C:\\Users\\15164\\Desktop\\nyc-taxis-vs-covid\\data\\taxi_2019.csv')
taxi_2020 = pd.read_csv('C:\\Users\\15164\\Desktop\\nyc-taxis-vs-covid\\data\\taxi_2020.csv')

In [10]:
taxi_2019.head(2)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,tip_amount,tolls_amount,total_amount,day_of_week,Start_Zone,End_Zone
0,2019-03-22 08:22:39,2019-03-22 08:32:50,1.0,1.45,170,107,1.0,8.5,2.95,0.0,14.75,Friday,Murray Hill-Queens,Gravesend
1,2019-03-21 15:31:46,2019-03-21 15:55:19,1.0,2.00,186,163,1.0,15.0,1.50,0.0,20.80,Thursday,Port Richmond,Midtown South


In [14]:
taxi_2020.head(2)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,tip_amount,tolls_amount,total_amount,day_of_week,Start_Zone,End_Zone
0,2020-03-08 00:00:33,2020-03-08 00:11:14,2.0,1.06,148,4,1.0,8.0,2.36,0.0,14.16,Sunday,Madison,Arden Heights
1,2020-03-08 00:02:10,2020-03-08 00:06:24,1.0,0.73,74,41,2.0,5.0,0.00,0.0,6.30,Sunday,East Harlem South,Central Harlem North


# Question 1:
The first questions this analysis will answer are the following:

**1-** What was the most expensive trip before the pandemic and between what zones did it occur? 

**2-** What was the least expensive trip before the pandemic and where did it occur?

**3-** Conversely, what were the most and least expensive trips during the pandemic and between which zones did they occur?

### 1- What was the most expensive trip before the pandemic and between what zones?

In [16]:
most_expensive = taxi_2019.sort_values('total_amount', ascending=False)

In [21]:
most_expensive.head(1)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,tip_amount,tolls_amount,total_amount,day_of_week,Start_Zone,End_Zone
2036968,2019-03-15 19:00:45,2019-03-15 19:12:48,1.0,0.0,233,79,3.0,943274.8,141492.02,0.0,1084772.17,Friday,Union Sq,East Williamsburg


As we can see above, according to our data **the most expensive trip in 2019 occured on March 15th and was for a total amount of a whopping $1,084,772.17. It was between Union Square and East Williamsburg.**

### 2 - What was the least expensive trip before the pandemic and where did it occur?

In [26]:
least_expensive = taxi_2019.loc[taxi_2019['total_amount'] > 0]

In [27]:
least_expensive = least_expensive.sort_values('total_amount', ascending=True)

In [4]:
len(least_expensive)

NameError: name 'least_expensive' is not defined

In [28]:
least_expensive.head(1)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,tip_amount,tolls_amount,total_amount,day_of_week,Start_Zone,End_Zone
6049675,2019-04-01 07:52:30,2019-04-01 07:52:30,1.0,0.0,264,264,2.0,0.01,0.0,0.0,0.01,Monday,NaN,NaN
